In [1]:
import pandas as pd
import fastparquet
import sqlite3

# This notebook has more tests of the database interaction. This was developed into database_interaction.py

In [7]:
# Setup a test file with animal information and a diet ID
def assign_diet_id(row):
    return f"{row['sampleId']}_{row['reportDate']}"

input_data = pd.read_parquet('../test_files/merged_cow_with_feed_daily_20230530_1557.parquet')
animal_data = input_data.dropna(subset=['sampleId'])

# Assign Diet_ID
animal_data = animal_data.assign(
        Diet_ID = lambda df: df['sampleId'] + '_' + df['reportDate'].dt.strftime('%Y-%m-%d')
        )

animal_data = animal_data.drop(columns=animal_data.columns[22:49], axis=1)


In [6]:
# Use Diet_ID to get the corresponding NDF value


conn = sqlite3.connect('../diet_database.db')
cursor = conn.cursor()

diet_id = 'DAIRY - LACTATING TMR_2021-01-25 00:00:00'  # Replace 'your_diet_id' with the actual Diet_ID value you want to retrieve

query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
cursor.execute(query)

column_data = cursor.fetchone()
column_data = float(column_data[0])
print(column_data)  # Display the retrieved column data

conn.close()

30.28


In [9]:
# Excecute the above code using a function that can be applied to each row of a dataframe
# This is a good example of how to pull individual values from the database using the Diet_ID to call the
# appropriate diet

def retrieve_ndf_value(row, conn):
    cursor = conn.cursor()

    diet_id = row['Diet_ID'] 

    query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
    cursor.execute(query)

    column_data = cursor.fetchone()
    ndf_value = float(column_data[0])  

    return ndf_value

conn = sqlite3.connect('../diet_database.db')

# Apply the function to each row of the dataframe and assign the result to a new column
animal_data['NDF_%_DM'] = animal_data.apply(retrieve_ndf_value, args=(conn,), axis=1)

conn.close()

In [12]:
# This function will retrieve the NDF (% DM) for the diet being fed and then multiply it by the 
# cow's DMI to give the NDF intake in kg (DM), which is needed for the model

def retrieve_ndf_value(row, conn):
    cursor = conn.cursor()

    diet_id = row['Diet_ID'] 

    query = f"SELECT `Neutral Detergent Fibre (%)` FROM current_diets WHERE Diet_ID = '{diet_id}'"
    cursor.execute(query)

    column_data = cursor.fetchone()
    # Divide by 100 to convert to a percentage
    ndf_DM = float(column_data[0])/100

    # Multiply by DMI to get the kg NDF consumed
    ndf_value = ndf_DM * row['DMI']
    return ndf_value

conn = sqlite3.connect('../diet_database.db')

# Apply the function to each row of the dataframe and assign the result to a new column
animal_data['NDF_kg_DM'] = animal_data.apply(retrieve_ndf_value, args=(conn,), axis=1)

conn.close()


In [9]:
conn = sqlite3.connect('../diet_database.db')
query = "SELECT Diet_ID, `Neutral Detergent Fibre (%)` FROM current_diets"
# create data frame from query:
df_NDF = pd.read_sql_query(query, conn)
conn.close()

# merge data:
animal_data_2 = pd.merge(
    animal_data,
    df_NDF,
    on = 'Diet_ID'
)

animal_data_2 = animal_data_2.assign(
    Dt_NDFIn = lambda df: df['Neutral Detergent Fibre (%)']/100 * df['DMI']
)